___
___
# **Proyecto Final Data Science**
## **Modelo de asociación**

Equipo: 2 - Quantum Insights
Integrantes:
- Felipe Varela - Product Owner
- Freddy Yaquive - Data Scientist
- Ivan Martinez - Data Scientist
- Sebastian Moya - Data Scientist
- Nicolás Lazarte - Scrum Master

Cohorte: DSFT01
___
___

In [16]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
import mlflow
import os
import pickle 

ruta_actual = os.getcwd()
mlflow.set_tracking_uri(f"file:///{ruta_actual}/mlruns")
mlflow.set_experiment("Modelos de Recomendación - Comparativa")

df_order_items = pd.read_csv("../databases/order_items.csv")
df_orders = pd.read_csv("../databases/orders.csv")
df_products = pd.read_csv("../databases/products.csv")

In [17]:
df_order_items = df_order_items.drop(columns="Unnamed: 0",axis=1)
df_orders = df_orders.drop(columns="Unnamed: 0",axis=1)
df_products = df_products.drop(columns="Unnamed: 0",axis=1)

In [18]:
# Se junta las tablas orders y order_items, para su entrenamiento de asociacion 
df_orders_final = df_order_items.merge(df_orders, how="left")

### Transformacion de tablas para 0 y 1, filas = usuarios, columnas = productos

In [19]:
from sklearn.model_selection import train_test_split

# Dividimos las interacciones originales
train_data, test_data = train_test_split(df_orders_final, test_size=0.2, random_state=42)

print(f"Registros totales: {len(df_orders_final)}")
print(f"Registros para entrenamiento: {len(train_data)}")
print(f"Registros para evaluación: {len(test_data)}")

Registros totales: 64359
Registros para entrenamiento: 51487
Registros para evaluación: 12872


In [20]:
# Creamos la matriz de utilidad usando solo los datos de entrenamiento
df_ratings_train = train_data.groupby(['user_id', 'product_id']).size().reset_index(name='purchase_count')
matriz_utilidad = df_ratings_train.pivot(index='user_id', columns='product_id', values='purchase_count')
matriz_utilidad = matriz_utilidad.fillna(0)

print(f"Matriz de entrenamiento creada. Dimensiones: {matriz_utilidad.shape}")

Matriz de entrenamiento creada. Dimensiones: (8618, 2000)


In [21]:
from sklearn.decomposition import TruncatedSVD

# n_components=20 es un buen punto de partida para este tamaño de catálogo
n_components = 20
svd = TruncatedSVD(n_components=n_components, random_state=42)

# Entrenamos el modelo sobre la matriz de utilidad
matriz_comprimida = svd.fit_transform(matriz_utilidad)

# Varianza explicada: Nos dice qué tanta información logramos conservar
varianza_total = svd.explained_variance_ratio_.sum()
print(f"Varianza explicada con {n_components} componentes: {varianza_total:.2%}")

Varianza explicada con 20 componentes: 29.53%


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# 1. Limpiamos y preparamos los nombres
df_products['ProductName'] = df_products['ProductName'].fillna('')

# 2. Vectorizamos los nombres (NLP)
tfidf = TfidfVectorizer(stop_words='english') # O 'spanish' si tu data es en español
tfidf_matrix = tfidf.fit_transform(df_products['ProductName'])

# 3. Calculamos la similitud de contenido (Coseno)
cos_sim_nlp = linear_kernel(tfidf_matrix, tfidf_matrix)

# 4. Convertimos a DataFrame para fácil acceso
df_sim_nlp = pd.DataFrame(
    cos_sim_nlp, 
    index=df_products['product_id'], 
    columns=df_products['product_id']
)

print("✅ Matriz de similitud NLP (Content-Based) lista.")

✅ Matriz de similitud NLP (Content-Based) lista.


In [23]:
from sklearn.metrics.pairwise import cosine_similarity

# Extraemos la matriz de productos (transpuesta de los componentes de SVD)
# svd.components_ tiene forma (n_components, n_productos)
matriz_productos = svd.components_.T

# Calculamos la similitud del coseno entre productos
similitud_productos = cosine_similarity(matriz_productos)

# Lo convertimos a DataFrame para facilitar la búsqueda
df_similitud = pd.DataFrame(
    similitud_productos, 
    index=matriz_utilidad.columns, 
    columns=matriz_utilidad.columns
)

print("Matriz de similitud entre productos lista.")

Matriz de similitud entre productos lista.


In [24]:
def recomendar_productos(product_id, df_sim, df_products, top_n=3):
    if product_id not in df_sim.index:
        return []

    categoria = df_products.loc[df_products['product_id'] == product_id, 'Category'].values[0]

    similares = df_sim[product_id].drop(product_id, errors='ignore').sort_values(ascending=False)

    productos_categoria = df_products[df_products['Category'] == categoria]['product_id'].values

    similares = similares[similares.index.isin(productos_categoria)]

    return similares.head(top_n).index.tolist()

In [25]:
id_a_probar = "P000001" 
ids_recomendados = recomendar_productos(id_a_probar, df_similitud, df_products)

nombre_base = df_products.loc[df_products['product_id'] == id_a_probar, 'ProductName'].values[0]
cat_base = df_products.loc[df_products['product_id'] == id_a_probar, 'Category'].values[0]

print(f"🔎 Producto base: {nombre_base} (Cat: {cat_base})")
print("-" * 60)

if not ids_recomendados:
    print("No se encontraron recomendaciones en la misma categoría.")
else:
    for i, p_id in enumerate(ids_recomendados, 1):
        nombre = df_products.loc[df_products['product_id'] == p_id, 'ProductName'].values[0]
        categoria = df_products.loc[df_products['product_id'] == p_id, 'Category'].values[0]
        URL = df_products.loc[df_products['product_id'] == p_id, 'Image_Url'].values[0]
        
        similitud = df_similitud.loc[id_a_probar, p_id]
        
        print(f"{i}. {nombre}")
        print(f"   ID: {p_id} | Cat: {categoria} | Similitud: {similitud:.4f}\n {URL}")
        print("-" * 30)

🔎 Producto base: Onion (Loose) (Cat: Fruits & Vegetables)
------------------------------------------------------------
1. Tomato - Local (Loose)
   ID: P000014 | Cat: Fruits & Vegetables | Similitud: 0.9992
 https://www.bigbasket.com/media/uploads/p/l/10000203_16-fresho-tomato-local.jpg
------------------------------
2. Carrot - Orange (Loose)
   ID: P000015 | Cat: Fruits & Vegetables | Similitud: 0.9865
 https://www.bigbasket.com/media/uploads/p/l/10000072_16-fresho-carrot-orange.jpg
------------------------------
3. Coconut - Large
   ID: P000241 | Cat: Fruits & Vegetables | Similitud: 0.9731
 https://www.bigbasket.com/media/uploads/p/l/10000092_15-fresho-coconut-large.jpg
------------------------------


In [26]:
ruta_actual = os.getcwd()

# Identificar la carpeta correcta donde se guardarán los modelos
if os.path.exists(os.path.join(ruta_actual, "modelos_entrenados")):
    ruta_modelos = os.path.join(ruta_actual, "modelos_entrenados")
else:
    # Buscar la carpeta subiendo un nivel si no está en el actual
    ruta_modelos = os.path.join(os.path.dirname(ruta_actual), "modelos_entrenados")

# Crear la carpeta automáticamente si no existe
if not os.path.exists(ruta_modelos):
    os.makedirs(ruta_modelos)

# Construir la ruta completa con el nombre del archivo final
archivo_salida = os.path.join(ruta_modelos, "modelo_svd_similitud.pkl")

# Guardar la tabla de similitudes en un archivo físico
with open(archivo_salida, 'wb') as f:
    pickle.dump(df_similitud, f) 

print(f"💾 Modelo guardado exitosamente en:\n{archivo_salida}")

💾 Modelo guardado exitosamente en:
c:\Users\fredd\Desktop\Soy Henry\ProyectoFinal\PF-Quantum_Insights\modelos_entrenados\modelo_svd_similitud.pkl


In [27]:
# Cálculo de Average Precision (AP) para una consulta
def compute_ap(predicciones, objetivos, k):
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicciones):
        if p in objetivos:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    return score / min(len(objetivos), k)

# Cálculo de NDCG - que tan cerca se esta de un ranking ideal
def compute_ndcg(predicciones, objetivos, k):
    dcg = 0.0
    idcg = 0.0
    for i, p in enumerate(predicciones):
        if p in objetivos:
            dcg += 1.0 / np.log2(i + 2)
    num_posibles = min(len(objetivos), k)
    for i in range(num_posibles):
        idcg += 1.0 / np.log2(i + 2)
    
    return dcg / idcg if idcg > 0 else 0.0

In [28]:
def metricas_svd(df_test, df_sim, df_products, k):
    ordenes_test = df_test.groupby('order_id')['product_id'].apply(list)
    metrics = {
        'Precision': [], 'Recall': [], 'F1': [],
        'MRR': [], 'MAP': [], 'NDCG': []
    }

    for items in ordenes_test:
        if len(items) < 2: continue
            
        for i in range(len(items)):
            semilla = items[i]
            objetivos = set(items[:i] + items[i+1:])
            predicciones = recomendar_productos(semilla, df_sim, df_products, top_n=k)
            
            if not predicciones: continue

            # --- CALCULOS ---
            aciertos = len(set(predicciones) & objetivos)
            
            # Precision & Recall
            prec = aciertos / k
            rec = aciertos / len(objetivos)
            metrics['Precision'].append(prec)
            metrics['Recall'].append(rec)
            
            # F1-Score
            if (prec + rec) > 0:
                f1 = 2 * (prec * rec) / (prec + rec)
            else:
                f1 = 0.0
            metrics['F1'].append(f1)
            
            # MRR
            rank_score = 0
            for rank, p_id in enumerate(predicciones, 1):
                if p_id in objetivos:
                    rank_score = 1 / rank
                    break
            metrics['MRR'].append(rank_score)
            
            # MAP & NDCG
            metrics['MAP'].append(compute_ap(predicciones, objetivos, k))
            metrics['NDCG'].append(compute_ndcg(predicciones, objetivos, k))

    # --- REPORTE ---
    print("\n" + "="*50)
    print(f"📊 REPORTE FINAL SVD (K={k})")
    print("="*50)
    print(f"{'Metrica':<15} | {'Valor':<10}")
    print("-" * 50)
    print(f"{'Precision':<15} | {np.mean(metrics['Precision']):.4f}")
    print(f"{'Recall':<15} | {np.mean(metrics['Recall']):.4f}")
    print(f"{'F1-Score':<15} | {np.mean(metrics['F1']):.4f}")
    print("-" * 50)
    print(f"{'MRR':<15} | {np.mean(metrics['MRR']):.4f}")
    print(f"{'MAP':<15} | {np.mean(metrics['MAP']):.4f}")
    print(f"{'NDCG':<15} | {np.mean(metrics['NDCG']):.4f}")
    print("="*50)
    name = f"SVD_K{k}"
    with mlflow.start_run(run_name=name):

        # --- Parámetros ---
        mlflow.log_param("modelo", "SVD")
        mlflow.log_param("k", k)

        # --- Métricas ---
        mlflow.log_metric("precision", np.mean(metrics['Precision']))
        mlflow.log_metric("recall", np.mean(metrics['Recall']))
        mlflow.log_metric("f1", np.mean(metrics['F1']))
        mlflow.log_metric("mrr", np.mean(metrics['MRR']))
        mlflow.log_metric("map", np.mean(metrics['MAP']))
        mlflow.log_metric("ndcg", np.mean(metrics['NDCG']))
    return metrics

resultados_svd = metricas_svd(test_data, df_similitud, df_products, k=5)


📊 REPORTE FINAL SVD (K=5)
Metrica         | Valor     
--------------------------------------------------
Precision       | 0.0120
Recall          | 0.0546
F1-Score        | 0.0195
--------------------------------------------------
MRR             | 0.0441
MAP             | 0.0403
NDCG            | 0.0448
